# 데이터 증강
아래의 방법들을 사용해 데이터 증강을 수행
- Round Trip Translation
- koEDA

In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv("../../data/train.csv")

### Round Trip Translation
kortt 라이브러리를 사용해 RTT를 수행했으며, papago로 번역하기에 naver CLIENT_ID와 SECRET_ID를 발급해야 한다.

In [4]:
from kortt import Generator

generator = Generator(mode="papago", client_id="BGj2lvXHjZhEVS0Xmb61", secret_id="ZOFUc7NxY5")

result = generator.generate("이뤄 뢰구긴 관광개 길보니 한구긔 이배…격차 계소 콱때되나")
print(result)

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (3.0.4)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(


긴 관광길이고, 아주 긴 관광선인데...그럴 가능성이 있나요


train data 10개만 증강

In [5]:
i = 0
for idx, data in enumerate(train_data["text"]):
    translated_data = generator.generate(data)
    train_data.loc[len(train_data)+1] = [train_data.iloc[idx, 0], translated_data, train_data.iloc[idx, 2], train_data.iloc[idx, 3], train_data.iloc[idx, 4]]
    i += 1
    if i == 10:
        break

In [6]:
train_data[7000:]

,ID,text,target,url,date
7001,ynat-v1_train_00000,개포2단지 분양 앞두고 개포지구 재건축 불붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37
7002,ynat-v1_train_00001,삼성전자 'KBIS 2018 셰프컬렉션' 선보여,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.10. 오전 8:33
7003,ynat-v1_train_00002,"LG G6를 사면 BO 이어폰은 5,000원 밖에 안 된다",0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.30. 오전 10:00
7004,ynat-v1_train_00003,새로운 블록체인 혁명 2030과 남성의 외로움,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.13. 오전 11:49
7005,ynat-v1_train_00004,이스라엘 정보당국은 테러 혐의로 팔레스타인인 50명을 체포했다,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.18. 오후 11:15
7006,ynat-v1_train_00005,배구연맹 이영일구 순천 KOVO 커 부녕 에이전시 입찰 공고,5,https://sports.news.naver.com/news.nhn?oid=001...,2019.07.10 16:47
7007,ynat-v1_train_00006,콜마비앤에이치 지난해 영업이익 352억 원...54.5%↑,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.01.18. 오후 4:45
7008,ynat-v1_train_00007,SOL 1조2000억원 경유·납품·항공유 공급계약 체결,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.05. 오전 10:21
7009,ynat-v1_train_00008,"카카오와 삼성화재, 디지털 손해보험사 설립 추진",1,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.10.23. 오전 10:49
7010,ynat-v1_train_00009,"김병준, 나경원 비대위 참석",6,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.02.08. 오후 6:32


papago를 사용해 RTT를 수행하니 생성되는 text의 quality가 좋았지만, papago의 일일사용량이 제한되어있음

### koEDA
koeda를 사용해 데이터를 증강했으며, 라이브러리의 기능은 아래와 같음
- AEasierDataAugmentation (AEDA) : 문장 안에 특수기호들을 랜덤으로 삽입
- RandomDeletion (RD) : p의 확률값을 가지고 문장 내의 각 단어들을 제거
- RandomInsertion (RI) : 하나의 단어를 선택하고 그 단어의 유의어를 문장내의 임의의 위치에 삽입. n번 시행
- SynonymReplacement (SR) : stop words가 아닌 단어들 중에서 랜덤으로 선택해서 유의어로 교체
- RandomSwap (RS) : 랜덤으로 2개의 단어를 선택해 위치를 바꾸며 n번 시행

In [241]:
from koeda import EDA, RD, RI, SR, RS 

In [251]:
# 각 args들은 문장 내에서 실행할 비율을 입력
eda = EDA(
    morpheme_analyzer="Okt", alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.0, prob_rd=0.0
)

random_delete = RD()    # 문장 내의 각 단어들을 무작위로 제거
random_insert = RI()    # 하나의 단어를 선택하고 그 단어의 유의어를 문장내의 임의의 위치에 삽입
random_swap = RS()      # 랜덤으로 2개의 단어를 선택해 위치를 바꿈
synonym_replace = SR()  # stop words가 아닌 단어들 중에서 랜덤으로 선택해 유의어로 교체

In [252]:
i = 0
for idx, data in enumerate(train_data["text"]):
    augmented_data1 = random_swap(data, p=0.2)
    augmented_data2 = random_insert(data, p=0.2)
    augmented_data3 = random_delete(data, p=0.2)
    augmented_data4 = synonym_replace(data, p=0.2)
    length = len(train_data)
    train_data.loc[length] = [train_data.iloc[idx, 0], augmented_data1, train_data.iloc[idx, 2], train_data.iloc[idx, 3], train_data.iloc[idx, 4]]
    train_data.loc[length+1] = [train_data.iloc[idx, 0], augmented_data2, train_data.iloc[idx, 2], train_data.iloc[idx, 3], train_data.iloc[idx, 4]]
    train_data.loc[length+2] = [train_data.iloc[idx, 0], augmented_data3, train_data.iloc[idx, 2], train_data.iloc[idx, 3], train_data.iloc[idx, 4]]
    train_data.loc[length+3] = [train_data.iloc[idx, 0], augmented_data4, train_data.iloc[idx, 2], train_data.iloc[idx, 3], train_data.iloc[idx, 4]]
    i += 1
    if i == 10:
        break

In [254]:
train_data[7000:]

,ID,text,target,url,date
7000,ynat-v1_train_00000,붙어2단지 분양개포앞두고 개포지구 재건축 불,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37
7001,ynat-v1_train_00000,개포 램프 2단지 분양 앞두고 개포지구 재건축 불붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37
7002,ynat-v1_train_00000,개포2단지 분양 앞두고 개포 재건축 불붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37
7003,ynat-v1_train_00000,개포2단지 분양 앞두고 개포부역 재건축 불붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37
7004,ynat-v1_train_00001,2018전자 KBIS 삼성서 셰프컬렉션 선보여,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.10. 오전 8:33
7005,ynat-v1_train_00001,삼성전자 KBIS 2018서 불손 셰프컬렉션 선보여,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.10. 오전 8:33
7006,ynat-v1_train_00001,삼성전자 KBIS 2018 셰프컬렉션 선보여,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.10. 오전 8:33
7007,ynat-v1_train_00001,삼성불손 KBIS 2018서 셰프컬렉션 선보여,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.10. 오전 8:33
7008,ynat-v1_train_00002,LG G이어폰 사면 BO 6이 단돈 5천원,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.30. 오전 10:00
7009,ynat-v1_train_00002,LG G6 사면 BO 이어폰이 단돈 5천원,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.04.30. 오전 10:00


In [188]:
train_data["text"][7000:]

7001            개포2단지불분양 앞두고 개포 지구재건축 붙어
7002           삼성2018 KBIS 전자서 셰프컬렉션 선보여
7003              LG G6 사면 BO 이 단돈이어폰5천원
7004              신간 블록체인혁명 2030고독사남자의 ·
7005        명당국 팔레스타인인 50정보 테러이스라엘혐의로 체포
7006    배구KOVO 이영일구 순천 연맹커 부녕 대행업체 공고 입찰
7007          마비…에이치 작년 영업익콜352억원앤54.5%↑
7008          SOil 1조2천억원 경유·납사공급·공유 항계약
7009            디지털·삼성화재 카카오 손해보험사 설립 추진
7010                 긴급비상대책위 참석하는나경원김병준과
Name: text, dtype: object

In [3]:
# 테스트
import sys
sys.path.append("/data/ephemeral/code/")
from preprocessing import data_augmentation

train_data = pd.read_csv("../../data/train.csv")

train_data = data_augmentation(train_data)

In [5]:
train_data[7000:]

,ID,text,target,url,date
7000,ynat-v1_train_00000,개포2단지 앞두고 개포지구 재건축 불붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37
7001,ynat-v1_train_00000,개포2단지 분양 앞두고 개포 병 지구 재건축 불붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37
7002,ynat-v1_train_00000,개포2단지 분양불앞두고 개포지구 재건축 붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37
7003,ynat-v1_train_00000,개포2단지 분양 앞두고 개포지대 재건축 불붙어,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.16. 오전 11:37
7004,ynat-v1_train_00001,전자 KBIS 2018서 셰프컬렉션 선보여,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.10. 오전 8:33
...,...,...,...,...,...
34995,ynat-v1_train_06998,문 대통령 1987 관람…깜짝 방문에 객석 환호·박수복합,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.07. 오후 5:23
34996,ynat-v1_train_06999,120년 전 대한제국으로…가을 정동에서 시간 여행,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.10.28. 오후 8:13
34997,ynat-v1_train_06999,120년 경각 전 대한제국으로…가을밤 정동에서 시간 여행 떠나다,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.10.28. 오후 8:13
34998,ynat-v1_train_06999,120년 전 대한제국으로…가을밤 떠나다에서 시간 여행 정동,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.10.28. 오후 8:13
